In [1]:
import sys
sys.path.append('/home/kasia/Python Projects/scrapping_die_borke')
import pandas as pd
import time
import sqlite3


import time
import requests
from bs4 import BeautifulSoup
from dateutil import parser
import re
import pandas as pd
import os.path
import sqlite3
 

In [9]:
url = 'https://www.surf-und-segelschule-mueggelsee.de/?Wetterdaten'

def download_weather_data(url):
    response = requests.get(url) # 
    html = response.content
    scraped = BeautifulSoup(html,'html.parser')
    return scraped

scraped= download_weather_data(url)

In [3]:
"""## save beaufitful soup to txt
file1 = open("../src/test/scraped_raw.txt","w")
file1.write(str(scraped))
file1.close()

## save list to txt

d = []
for data in scraped.find('div',class_="tbis-layout-cell layout-item-old-1").find_all('tr'):
        d.append(data.text)
file1 = open("../src/test/for_test_extract_float_from_string.txt","w")
file1.write(str(d))
file1.close()
"""
def extract_float_from_string(string):
    l = [float(x) for x in re.findall("\d+\.\d+", string)]
    if len(l) == 1:
        return l[0]
    else:
        return l



In [42]:

with open("../src/test/scraped_raw.txt",'r') as f:
    soup = BeautifulSoup(f, "html.parser")
parse_data(soup)

{'lastupdate': datetime.datetime(2020, 9, 16, 22, 8, 21),
 'temp': 21.4,
 'wind': 9.5,
 'boen': 17.3,
 'niederschlag': 0.0,
 'feuchte': 74,
 'luftdruck': 1018.1}

In [22]:
def parse_data(scraped):
    d = []
    for data in scraped.find('div',class_="tbis-layout-cell layout-item-old-1").find_all('tr'):
        d.append(data.text)
    mapping = {}
    mapping['lastupdate'] = parser.parse(d[0].split(': ')[1])
    mapping['temp'] = extract_float_from_string(d[2])
    mapping['wind'] = extract_float_from_string(d[3])[0]
    mapping['boen'] = extract_float_from_string(d[3])[1]
    mapping['niederschlag'] = extract_float_from_string(d[4])
    mapping['feuchte'] = int(re.findall("\d+\\d+", d[5])[0])
    mapping['luftdruck'] = extract_float_from_string(d[6])
    return mapping

In [ ]:
def create_db(database_name):
    connection = sqlite3.connect(database_name)
    cursor = connection.cursor()
    #cursor.execute('DROP TABLE IF EXISTS weather')
    sql_command = """CREATE TABLE IF NOT EXISTS weather (
    id INTEGER PRIMARY KEY,
    lastupdate DATETIME,
    temp REAL,
    wind REAL,
    boen REAL,
    niederschlag REAL,
    feuchte REAL,
    luftdruck REAL)"""

    cursor.execute(sql_command)
    connection.close()
    
def insert_to_db_table(mapping,database_name):
    connection = sqlite3.connect(database_name)
    cursor = connection.cursor()
    sql_command = """INSERT INTO weather 
    (lastupdate, temp, wind, boen, 
    niederschlag, feuchte, luftdruck) 
    VALUES (:lastupdate,:temp,:wind,:boen,
    :niederschlag,:feuchte,:luftdruck)"""

    cursor.execute(sql_command,mapping)
    connection.commit()
    connection.close()

In [ ]:
create_db('data/scrap_die_borke.db')
url = 'https://www.surf-und-segelschule-mueggelsee.de/?Wetterdaten'
while True:
    try:
        get_data(url,'data/scrap_die_borke.db')
        time.sleep(30)
    except Exception as e:
        print("File not accessible",e)

In [ ]:
connection = sqlite3.connect("data/scrap_die_borke.db")
pd.read_sql("SELECT * FROM weather",connection)


In [ ]:
def get_data(url):
    scraped = download_weather_data(url)
    mapping = parse_data(scraped)
    insert_to_db_table(mapping,'data/scrap_die_borke.db')
    
    
    
